In [30]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, models
import pandas as pd

In [2]:
labeled_data = pd.read_csv('train.csv')
unlabeled_data = pd.read_csv('test.csv')

In [3]:
labeled_data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
labels = labeled_data[["price_range"]]
labeled_data = labeled_data.drop('price_range', axis=1)
unlabeled_data = unlabeled_data.drop('id', axis=1)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(labeled_data, labels, test_size=0.2, random_state=42)

In [21]:
input_data = layers.Input(shape=(20,))
encoded = layers.Dense(64, activation='relu')(input_data)
decoded = layers.Dense(20, activation='sigmoid')(encoded)

In [22]:
autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [23]:
autoencoder.fit(unlabeled_data, unlabeled_data, epochs=100, batch_size=32)

Epoch 1/100
32/32 [==============================] - 0s 1ms/step - loss: 491310.1250
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 491308.1562
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.5625
Epoch 4/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.5312
Epoch 5/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.5938
Epoch 6/100
32/32 [==============================] - 0s 894us/step - loss: 491307.5625
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.5312
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.5938
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 491307.5938
Epoch 10/100
32/32 [==============================] - 0s 2ms/step - loss: 491307.5625
Epoch 11/100
32/32 [==============================] - 0s 1ms/step - loss: 491307.6250
Epoch 12/100
32/32 [==============================] - 0s 1ms/

In [25]:
encoder = models.Model(autoencoder.input, autoencoder.layers[-2].output)
encoded_labeled_data = encoder.predict(X_train)
encoded_unlabeled_data = encoder.predict(unlabeled_data)

32/32 [==============================] - 0s 848us/step


In [26]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(encoded_labeled_data, y_train)

C:\Users\abdur\AppData\Local\Temp\ipykernel_24528\825357849.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_classifier.fit(encoded_labeled_data, y_train)


RandomForestClassifier(random_state=42)

In [27]:
encoded_test_data = encoder.predict(X_test)

13/13 [==============================] - 0s 1ms/step


In [28]:
y_pred = rf_classifier.predict(encoded_test_data)

In [31]:
accuracy = accuracy_score(y_test, y_pred)
print("Test doğruluğu:", accuracy)

Test doğruluğu: 0.96
